This script generates the input TAM matrices of selected genes, which serve as input for the entire model in the second step of training. 

In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from pyfaidx import Fasta
from verstack import stratified_continuous_split
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
import scipy
from scipy import stats
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR
import h5py
import seaborn as sns
import os
os.chdir('/nfs/public/xixi/scRegulate/T2D')

2024-01-03 12:41:11.308195: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-03 12:41:11.514741: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-03 12:41:12.364191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-11.5/lib64
2024-01-03 12:41:12.364284: W tensorflow/compiler/xla/stream

In [2]:
genes = Fasta('../../ref_genome/hg19.fa')

In [3]:
df_x = pd.read_csv('./data/beta.atac.aggregate_30cells.csv', index_col=0).transpose()
df_x

,10-100001665-100002165,10-100002531-100003031,10-100003836-100004336,10-100005433-100005933,10-100006548-100007048,10-100013839-100014339,10-100018334-100018834,10-100019564-100020064,10-100020408-100020908,10-100021589-100022089,...,Y-8756349-8756849,Y-8861982-8862482,Y-8869228-8869728,Y-8894760-8895260,Y-8895712-8896212,Y-8902164-8902664,Y-8903250-8903750,Y-9490312-9490812,Y-9647600-9648100,Y-9649551-9650051
V1,0.000000,0.0,0.00000,0.468211,0.468211,0.936421,0.000000,0.468211,0.468211,0.468211,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V2,0.000000,0.0,0.00000,0.000000,0.468040,0.468040,0.000000,0.000000,0.468040,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V3,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.627410,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V4,0.565643,0.0,0.00000,0.000000,0.565643,0.000000,1.131286,0.896523,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V5,0.000000,0.0,0.00000,0.000000,0.000000,0.484623,0.000000,0.484623,0.000000,0.484623,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V1390,0.000000,0.0,0.00000,0.658323,0.000000,1.043417,0.000000,0.658323,0.658323,0.658323,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V1391,0.000000,0.0,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V1392,0.000000,0.0,0.00000,0.000000,0.000000,0.910113,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.910113,0.0,0.0,0.0,0.0,0.0,0.0,0.0
V1393,0.630010,0.0,0.63001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df_y = pd.read_csv('./data/beta.rna.aggregate_30cells.csv', index_col=0).transpose()
df_y

,WASH7P,RP11-34P13.7,RP11-34P13.10,AL627309.1,RP11-34P13.13,RP11-34P13.9,AP006222.2,RP11-206L10.17,RP5-857K21.2,RP5-857K21.9,...,CH507-513H4.6,CH17-408M7.1,CH507-39O4.1,CH507-39O4.2,CH507-24F1.1,CH507-338C24.1,CH507-254M2.3,CH507-154B10.2,CH507-145C22.3,CH17-351M24.1
V1,0.0,0.515306,0.0,0.515306,0.515306,0.0,0.515306,1.446647,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.515306,0.816741,0.0,0.000000
V2,0.0,0.494710,0.0,0.494710,0.784097,0.0,0.494710,0.784097,0.0,0.0,...,0.0,0.000000,0.0,0.494710,0.000000,1.278808,0.784097,1.148682,0.0,0.494710
V3,0.0,0.000000,0.0,0.000000,0.629183,0.0,0.000000,0.997232,0.0,0.0,...,0.0,0.000000,0.0,0.629183,0.000000,1.460918,0.629183,0.629183,0.0,0.000000
V4,0.0,0.610912,0.0,0.968272,0.610912,0.0,0.000000,0.968272,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.610912,1.418493,0.000000,0.0,0.000000
V5,0.0,0.499200,0.0,0.000000,0.000000,0.0,0.499200,1.159107,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,1.159107,0.499200,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
V1390,0.0,0.000000,0.0,0.000000,0.666096,0.0,1.055737,1.869967,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,1.546626,0.666096,1.055737,0.0,0.000000
V1391,0.0,1.166952,0.0,0.736265,0.000000,0.0,1.472530,2.333904,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,1.166952,0.736265,0.000000,0.0,0.000000
V1392,0.0,0.000000,0.0,0.788265,0.000000,0.0,1.830293,1.249370,0.0,0.0,...,0.0,0.788265,0.0,0.000000,0.000000,0.788265,1.576529,0.788265,0.0,0.000000
V1393,0.0,0.000000,0.0,0.505620,0.505620,0.0,1.011240,0.505620,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,1.011240,0.505620,0.000000,0.0,0.000000


In [5]:
df_peaks = pd.DataFrame(df_x.columns)[0].str.split('-',expand=True) 
df_peaks = df_peaks.rename(columns={0: "chrom", 1: "start", 2: "end"})
df_peaks["chrom"] = 'chr' + df_peaks["chrom"]
df_peaks["start"] = pd.to_numeric(df_peaks["start"])
df_peaks["end"] = pd.to_numeric(df_peaks["end"])
df_peaks

,chrom,start,end
0,chr10,100001665,100002165
1,chr10,100002531,100003031
2,chr10,100003836,100004336
3,chr10,100005433,100005933
4,chr10,100006548,100007048
...,...,...,...
412092,chrY,8902164,8902664
412093,chrY,8903250,8903750
412094,chrY,9490312,9490812
412095,chrY,9647600,9648100


In [6]:
geneanno = pd.read_csv('../../ref_genome/hg19_geneanno.txt', sep='\t')
geneanno = geneanno.drop_duplicates(subset=['Gene name'])
geneanno

,Gene stable ID,Transcript stable ID,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,Gene name,Transcription start site (TSS)
0,ENSG00000271782,ENST00000607815,1,50902700,50902978,-1,RP5-850O15.4,50902978
1,ENSG00000232753,ENST00000424955,1,103817769,103828355,1,RP11-347K2.1,103817769
2,ENSG00000225767,ENST00000424664,1,50927141,50936822,1,RP5-850O15.3,50927141
3,ENSG00000202140,ENST00000365270,1,50965430,50965529,-1,Y_RNA,50965529
4,ENSG00000207194,ENST00000384465,1,51048076,51048183,1,RNU6-1026P,51048076
...,...,...,...,...,...,...,...,...
179049,ENSG00000229926,ENST00000442569,9,141031547,141038316,1,RP11-424E7.3,141031547
179050,ENSG00000159247,ENST00000503395,9,141044565,141071821,1,TUBBP5,141044565
179053,ENSG00000237419,ENST00000428088,9,141090383,141093775,-1,RP11-885N19.6,141093775
179054,ENSG00000233013,ENST00000446912,9,141106637,141143444,1,FAM157B,141107518


In [7]:
motif_files = os.listdir('../../ref_genome/JASPAR_motifs_pfm_homosapiens/pfm.np')
motif_files[:5]

['MA0002.1.RUNX1.npy',
 'MA0003.4.TFAP2A.npy',
 'MA0007.2.AR.npy',
 'MA0014.3.PAX5.npy',
 'MA0009.2.TBXT.npy']

In [8]:
tfs_kept = []
tf_by_region_mat = []
for i in list(motif_files):
    tf = i.split('.')[-2]#.capitalize()
    if tf in df_y.columns:
        if tf in tfs_kept:
            continue
            
        tfs_kept.append(tf)
tfs_kept = sorted(tfs_kept)

In [10]:
files = os.listdir('/nfs/public/xixi/scRegulate/T2D/nn.best.feature6.learnW')
markers_filtered = []
for file in files:
    if len(file.split('.2.pt'))>1:
        marker = file.split('.2.pt')[0]
        if marker not in markers_filtered:
            markers_filtered.append(marker)
len(markers_filtered)

667

In [11]:
markers = pd.read_csv("./data/markers_status.txt", sep='\t')
markers = markers.groupby('cluster').apply(lambda x: x.nlargest(100, 'avg_log2FC')).reset_index(drop=True)
markers = markers['gene'].drop_duplicates().to_list()
len(markers)

297

In [12]:
genes_filtered = [i for i in markers_filtered if i not in tfs_kept]
print(len(genes_filtered))
genes_filtered = [i for i in genes_filtered if i in markers]
print(len(genes_filtered))

644
239


In [13]:
df = pd.read_csv("pearson.testset.feature6_learnW.txt", sep='\t', header=None, index_col=0)
df = df.iloc[:, :3]
df = df.loc[genes_filtered, :]
df

,1,2,3
0,,,
LDLR,0.789120,0.790108,0.790603
EML5,0.959619,0.960493,0.960958
MARK1,0.926361,0.928668,0.927640
EWSR1,0.888747,0.887867,0.889548
CTD-2587M2.1,0.686387,0.691965,0.683586
...,...,...,...
THSD7A,0.717883,0.716057,0.719220
DNAJA4,0.746805,0.794529,0.796820
RP11-3L8.3,0.803960,0.802976,0.799333


In [ ]:
def init_with_precomputed_matrix(tensor, precomputed_matrix):
    tensor.data = precomputed_matrix.clone()

class Net(nn.Module):
    def __init__(self, num_peak, num_tf, W):
        super(Net, self).__init__()
        self.num_peak = num_peak
        self.num_tf = num_tf

        self.W = nn.Parameter(torch.Tensor(num_tf, num_peak))
        init_with_precomputed_matrix(self.W, W)

        self.fc1 = nn.Linear(self.num_peak * self.num_tf, 1)
        self.fc1_activate = nn.ReLU()
        
    def forward(self, x):
        x = x * abs(self.W.repeat(x.shape[0], 1, 1))
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [38]:
# 初始化一个空的三维矩阵
X_all = []
expr_all = []
num_peaks = []
peaks_all = pd.DataFrame()
count = 0
for marker in genes_filtered:
    print(count)
    count = count+1
    length = 250000
    anno = geneanno.loc[geneanno['Gene name']==marker,:]
    if anno.shape[0]>0:
        print(marker)
        chrom = 'chr'+ str(anno['Chromosome/scaffold name'].values[0])
        start = anno['Gene start (bp)'].values[0]
        end = anno['Gene end (bp)'].values[0]
        strand = anno['Strand'].values[0]
        if strand==1:
            peaks = df_peaks.loc[(df_peaks['chrom']==chrom) & ((df_peaks['start']>=start-length) & (df_peaks['start']<start+length) | 
                                                               (df_peaks['end']>start-length) & (df_peaks['end']<=start+length)),:]
        else:
            peaks = df_peaks.loc[(df_peaks['chrom']==chrom) & ((df_peaks['start']>=end-length) & (df_peaks['start']<end+length) | 
                                                               (df_peaks['end']>end-length) & (df_peaks['end']<=end+length)),:]
        cellcount = (df_x.iloc[:, peaks.index]!=0).astype(int).sum(axis=1)
        cells = cellcount[cellcount!=0].index
    else:
        print(marker)
        print('No such gene in the gene annotation file.')
        continue


    y = np.array(df_y.loc[:, marker], dtype='float16').flatten()

    tfs_use = tfs_kept.copy()
    cell_by_tf_mat = np.array(df_y.loc[:, tfs_use])
    cell_by_peak_mat = np.array(df_x.iloc[:, peaks.index])
    order = np.argsort(np.array(peaks.loc[:,'start']))
    cell_by_peak_mat = cell_by_peak_mat[:, order]
    # Scale data
    cell_by_peak_mat = cell_by_peak_mat/np.max(cell_by_peak_mat)

    cell_by_tf_mat = np.repeat(cell_by_tf_mat[:, :, np.newaxis], peaks.shape[0], axis=2)
    cell_by_peak_mat = np.repeat(cell_by_peak_mat[:, np.newaxis, :], len(tfs_use), axis=1)
    X = cell_by_tf_mat * cell_by_peak_mat

    y_std = (y - y.min()) / (y.max() - y.min())
    y = y_std * (1 - (0)) + (0)
   
    print(X.shape)
    
    W = torch.ones(X.shape[1], X.shape[2])
    model = Net(num_peak=X.shape[2], num_tf=X.shape[1], W=W)
    num = np.argsort(np.array(df.loc[marker]))[::-1][0]
    model.load_state_dict(torch.load('./nn.best.feature6.learnW/'+marker+'.'+str(num)+'.pt', map_location='cpu'))
    W = abs(model.W.data).clone().detach().numpy()[:, order]
    X = (X*W).astype('float16')
    
    X_all.append(X)
    expr_all.append(y)
    peaks_all = peaks_all.append(peaks.iloc[order, :].reset_index(drop=True))
    num_peaks.append(X.shape[2])
    
print('All done!')

0
LDLR
(1394, 457, 183)
1
EML5
(1394, 457, 114)
2
MARK1
(1394, 457, 83)
3
EWSR1
(1394, 457, 139)
4
CTD-2587M2.1
(1394, 457, 90)
5
NPLOC4
(1394, 457, 151)
6
PAX8-AS1
(1394, 457, 104)
7
STX3
(1394, 457, 67)
8
KIF1A
(1394, 457, 188)
9
ABCA1
(1394, 457, 119)
10
HNRNPH1
(1394, 457, 163)
11
GOLGA4
(1394, 457, 154)
12
PLA2G4C
(1394, 457, 58)
13
USP34
(1394, 457, 144)
14
PI4KA
(1394, 457, 132)
15
SGSM1
(1394, 457, 110)
16
HNRNPM
(1394, 457, 128)
17
NEBL
(1394, 457, 120)
18
PER3
(1394, 457, 133)
19
SMURF1
(1394, 457, 156)
20
FAM168B
(1394, 457, 82)
21
AGPAT5
(1394, 457, 128)
22
ATG13
(1394, 457, 156)
23
HSP90B1
(1394, 457, 106)
24
GRAMD3
(1394, 457, 99)
25
SUPT6H
(1394, 457, 138)
26
TMEM39A
(1394, 457, 99)
27
FUS
(1394, 457, 115)
28
HTT
(1394, 457, 182)
29
SRSF11
(1394, 457, 65)
30
GGA2
(1394, 457, 99)
31
TXNRD1
(1394, 457, 116)
32
MOK
(1394, 457, 134)
33
SPIRE1
(1394, 457, 142)
34
IFT88
(1394, 457, 58)
35
RBM5
(1394, 457, 134)
36
RAB1A
(1394, 457, 166)
37
KIAA0368
(1394, 457, 111)
38
NEK10
(13

In [39]:
X = np.concatenate(X_all, axis=2)
X.shape

(1394, 457, 25198)

In [41]:
expr = np.array(expr_all).T
expr.shape

(1394, 239)

In [42]:
s = 0
for i in num_peaks:
    s = s + i
print(s)

25198


In [44]:
peaks_all

,chrom,start,end
0,chr19,10954859,10955359
1,chr19,10958800,10959300
2,chr19,10960632,10961132
3,chr19,10962809,10963309
4,chr19,10964368,10964868
...,...,...,...
62,chr8,53854105,53854605
63,chr8,53878345,53878845
64,chr8,53883166,53883666
65,chr8,53886237,53886737


In [45]:
peaks_all['chrom'] = peaks_all['chrom'].astype(str)
peaks_all['start'] = peaks_all['start'].astype(str)
peaks_all['end'] = peaks_all['end'].astype(str)

In [46]:
h5f = h5py.File('/nfs/public/xixi/scRegulate/T2D/predict_status/data_T2D_float16.h5', 'w')
h5f.create_dataset('X', data=X)
h5f.create_dataset('expr', data=expr)
h5f.create_dataset('num_peaks', data=np.array(num_peaks))
h5f.create_dataset('peaks', data=peaks_all.to_numpy())
h5f.close()